In [1]:
import pandas as pd
import numpy as np
import datetime
from scipy.stats import pearsonr

#### Importing Datasets

In [2]:
# Import 2021 + 2022 CAHOOTS data from a XLSX
raw_data_2021_2022 = pd.read_excel('raw_data/call_data_from_CAHOOTS_2021_2022.xlsx')

# Import 2023 CAHOOTS data from a CSV
raw_data_2023 = pd.read_excel('raw_data/call_data_from_CAHOOTS.xlsx')

# Combining both data sets vertically
raw_data = pd.concat([raw_data_2021_2022, raw_data_2023], ignore_index=True)


# Function to remove date part from datetime values
def remove_date_from_datetime(time_value):
    return time_value.time()

# Add a new column specifying the type of each value in 'TimeOfCall'
raw_data['TimeOfCall_Type'] = raw_data['TimeOfCall'].apply(lambda x: type(x).__name__)

# Apply the logic to remove the date part if the value type is 'datetime'
raw_data['TimeOfCall'] = raw_data.apply(lambda row: remove_date_from_datetime(row['TimeOfCall']) if row['TimeOfCall_Type'] == 'datetime' else row['TimeOfCall'], axis=1)

# Function to convert value to time format
def convert_to_time(value):
    if isinstance(value, pd.Timestamp):  # Check if value is a datetime
        return value.time()
    elif isinstance(value, str):  # Check if value is a string
        try:
            return pd.to_datetime(value, format='%H:%M:%S').time()
        except ValueError:
            return value  # Return the original value if parsing fails
    else:
        return value

# Convert 'TimeOfCall' column to consistent time format
raw_data['TimeOfCall'] = raw_data['TimeOfCall'].apply(convert_to_time)
raw_data

,Date,TimeOfCall,Age,Gender,Race,Language,City,Reason for Dispatch,TimeOfCall_Type
0,2021-01-04,20:10:31,48,Female,White,English,Eugene,Public Assist,time
1,2021-01-14,14:28:00,34,Male,White,English,Springfield,Public Assist,time
2,2021-01-14,14:55:00,Unavailable,Chose not to Disclose,Chose not to Disclose,English,Springfield,Check Welfare,time
3,2021-01-14,14:53:00,Unavailable,Chose not to Disclose,Chose not to Disclose,English,Springfield,Check Welfare,time
4,2021-01-14,16:11:00,35,Male,White,English,Springfield,Check Welfare,time
...,...,...,...,...,...,...,...,...,...
67516,2023-12-31,21:33:35,Not Given,Chose not to disclose,White,English,Springfield,Information Not Available,datetime
67517,2023-12-31,21:51:35,Not Given,Chose not to disclose,NaN,English,Informatio not recorded,Information Not Available,datetime
67518,2023-12-31,21:58:10,17,Chose not to disclose,White,English,Eugene,Information Not Available,datetime
67519,2023-12-31,22:15:00,17,Chose not to disclose,White,English,Eugene,Information Not Available,datetime


In [3]:
# Import Eugene climate 21-22 data from a CSV
eugene_climate_data_21_22 = pd.read_csv('raw_data/weather_database_21_22.csv')

# Import Eugene climate 23 data from a CSV
eugene_climate_data_23 = pd.read_csv('raw_data/weather_database_23.csv')

# Vertically merge the two DataFrames
eugene_climate_data = pd.concat([eugene_climate_data_21_22, eugene_climate_data_23], ignore_index=True)

# Rename the 'datetime' column to 'date'
eugene_climate_data = eugene_climate_data.rename(columns={'datetime': 'date'})

# Save the merged DataFrame to a CSV
eugene_climate_data.to_csv('data/merged_eugene_climate_data.csv', index=False)
eugene_climate_data.head()
eugene_climate_data.columns

Index(['name', 'date', 'tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
       'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations'],
      dtype='object')

In [4]:
# Import AQI data from a CSV
aqi_data = pd.read_csv('raw_data/eugene-- highway 99, oregon-air-quality.csv')
# Rename the pm25 column
aqi_data = aqi_data.rename(columns={' pm25': 'pm25'})
print(aqi_data.shape)
aqi_data.head()

(3354, 2)


,date,pm25
0,2024/4/2,31
1,2024/4/3,25
2,2024/4/4,11
3,2024/4/5,15
4,2024/4/6,17


#### Importing Upcoming Weather Data

In [5]:
# Import AQI data from a CSV
upcoming_data = pd.read_csv('raw_data/upcoming_weather.csv')
# Rename the 'datetime' column to 'date'
upcoming_data = upcoming_data.rename(columns={'datetime': 'date'})
upcoming_data.head()

,name,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"Eugene, OR, United States",2024-06-04,69.0,53.0,60.2,69.0,53.0,60.2,52.8,77.0,...,8.0,4,10,2024-06-04T05:30:47,2024-06-04T20:51:22,0.94,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KCVO,WWCO3,KEUG"
1,"Eugene, OR, United States",2024-06-05,76.0,53.0,62.2,76.0,53.0,62.2,51.4,71.0,...,28.5,9,10,2024-06-05T05:30:25,2024-06-05T20:52:05,0.97,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,NaN
2,"Eugene, OR, United States",2024-06-06,80.0,49.0,64.6,80.0,49.0,64.6,48.5,58.9,...,31.0,10,10,2024-06-06T05:30:06,2024-06-06T20:52:45,0.00,Clear,Clear conditions throughout the day.,clear-day,NaN
3,"Eugene, OR, United States",2024-06-07,84.3,53.7,68.8,83.4,53.7,68.8,54.2,61.9,...,30.1,9,10,2024-06-07T05:29:48,2024-06-07T20:53:24,0.04,Clear,Clear conditions throughout the day.,clear-day,NaN
4,"Eugene, OR, United States",2024-06-08,78.9,54.9,67.1,78.9,54.9,67.1,52.9,62.3,...,28.0,9,10,2024-06-08T05:29:33,2024-06-08T20:54:01,0.07,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,NaN


#### Adding a Predicted pm25 value to the Upcoming Data

In [6]:
# Convert 'date' columns to datetime
aqi_data['date'] = pd.to_datetime(aqi_data['date'])
upcoming_data['date'] = pd.to_datetime(upcoming_data['date'])

# Calculate average pm25 for each month in 'aqi_data'
aqi_data['month'] = aqi_data['date'].dt.month
monthly_avg_pm25 = aqi_data.groupby(aqi_data['month'])['pm25'].mean()

# Create a new column in 'upcoming_data' to store the average pm25
upcoming_data['pm25'] = upcoming_data['date'].dt.month.map(monthly_avg_pm25)
upcoming_data

,name,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations,pm25
0,"Eugene, OR, United States",2024-06-04,69.0,53.0,60.2,69.0,53.0,60.2,52.8,77.0,...,4,10,2024-06-04T05:30:47,2024-06-04T20:51:22,0.94,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KCVO,WWCO3,KEUG",16.47037
1,"Eugene, OR, United States",2024-06-05,76.0,53.0,62.2,76.0,53.0,62.2,51.4,71.0,...,9,10,2024-06-05T05:30:25,2024-06-05T20:52:05,0.97,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,NaN,16.47037
2,"Eugene, OR, United States",2024-06-06,80.0,49.0,64.6,80.0,49.0,64.6,48.5,58.9,...,10,10,2024-06-06T05:30:06,2024-06-06T20:52:45,0.00,Clear,Clear conditions throughout the day.,clear-day,NaN,16.47037
3,"Eugene, OR, United States",2024-06-07,84.3,53.7,68.8,83.4,53.7,68.8,54.2,61.9,...,9,10,2024-06-07T05:29:48,2024-06-07T20:53:24,0.04,Clear,Clear conditions throughout the day.,clear-day,NaN,16.47037
4,"Eugene, OR, United States",2024-06-08,78.9,54.9,67.1,78.9,54.9,67.1,52.9,62.3,...,9,10,2024-06-08T05:29:33,2024-06-08T20:54:01,0.07,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,NaN,16.47037
5,"Eugene, OR, United States",2024-06-09,75.5,54.2,64.5,75.5,54.2,64.5,51.6,64.6,...,9,10,2024-06-09T05:29:20,2024-06-09T20:54:37,0.11,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,NaN,16.47037
6,"Eugene, OR, United States",2024-06-10,76.7,51.5,64.2,76.7,51.5,64.2,49.4,61.2,...,10,10,2024-06-10T05:29:10,2024-06-10T20:55:10,0.14,Clear,Clear conditions throughout the day.,clear-day,NaN,16.47037
7,"Eugene, OR, United States",2024-06-11,77.3,50.1,61.8,77.3,50.1,61.8,50.3,67.0,...,9,10,2024-06-11T05:29:01,2024-06-11T20:55:42,0.17,Clear,Clear conditions throughout the day.,clear-day,NaN,16.47037


#### Building a Dataframe with all of our Data (minus Upcoming)

In [7]:
# Convert the 'date' column in eugene_climate_data to datetime64[ns]
eugene_climate_data['date'] = pd.to_datetime(eugene_climate_data['date'])

# Convert the 'date' column in aqi_data to datetime64[ns]
aqi_data['date'] = pd.to_datetime(aqi_data['date'])

# Merge eugene_climate_data and aqi_data on 'date'
eugene_climate_data_aqi = pd.concat([eugene_climate_data.set_index('date'), aqi_data.set_index('date')], axis=1, join='outer')

# Reset index to make 'date' a column again
eugene_climate_data_aqi = eugene_climate_data_aqi.reset_index()

In [8]:
# Convert 'Date' column in raw_data to datetime
raw_data['Date'] = pd.to_datetime(raw_data['Date'])

# Merge with Eugene climate data
raw_all_col_data = pd.merge(raw_data, eugene_climate_data_aqi, left_on='Date', right_on='date', how='left')

# Drop redundant columns
raw_all_col_data.drop(['date'], axis=1, inplace=True)

# Display the merged DataFrame
raw_all_col_data

,Date,TimeOfCall,Age,Gender,Race,Language,City,Reason for Dispatch,TimeOfCall_Type,name,...,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations,pm25,month
0,2021-01-04,20:10:31,48,Female,White,English,Eugene,Public Assist,time,"Eugene, OR, United States",...,NaN,2021-01-04T07:47:20,2021-01-04T16:48:02,0.70,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"KCVO,72693024221,99999904236,72694524202,F6274...",13.0,1.0
1,2021-01-14,14:28:00,34,Male,White,English,Springfield,Public Assist,time,"Eugene, OR, United States",...,NaN,2021-01-14T07:44:22,2021-01-14T16:59:14,0.05,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KCVO,WWCO3,72693024221,99999904236,72694524202...",24.0,1.0
2,2021-01-14,14:55:00,Unavailable,Chose not to Disclose,Chose not to Disclose,English,Springfield,Check Welfare,time,"Eugene, OR, United States",...,NaN,2021-01-14T07:44:22,2021-01-14T16:59:14,0.05,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KCVO,WWCO3,72693024221,99999904236,72694524202...",24.0,1.0
3,2021-01-14,14:53:00,Unavailable,Chose not to Disclose,Chose not to Disclose,English,Springfield,Check Welfare,time,"Eugene, OR, United States",...,NaN,2021-01-14T07:44:22,2021-01-14T16:59:14,0.05,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KCVO,WWCO3,72693024221,99999904236,72694524202...",24.0,1.0
4,2021-01-14,16:11:00,35,Male,White,English,Springfield,Check Welfare,time,"Eugene, OR, United States",...,NaN,2021-01-14T07:44:22,2021-01-14T16:59:14,0.05,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KCVO,WWCO3,72693024221,99999904236,72694524202...",24.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67516,2023-12-31,21:33:35,Not Given,Chose not to disclose,White,English,Springfield,Information Not Available,datetime,"Eugene, OR, United States",...,10.0,2023-12-31T07:47:19,2023-12-31T16:43:40,0.66,Overcast,Cloudy skies throughout the day.,fog,"KCVO,WWCO3,72693024221,72694524202,KEUG",28.0,12.0
67517,2023-12-31,21:51:35,Not Given,Chose not to disclose,NaN,English,Informatio not recorded,Information Not Available,datetime,"Eugene, OR, United States",...,10.0,2023-12-31T07:47:19,2023-12-31T16:43:40,0.66,Overcast,Cloudy skies throughout the day.,fog,"KCVO,WWCO3,72693024221,72694524202,KEUG",28.0,12.0
67518,2023-12-31,21:58:10,17,Chose not to disclose,White,English,Eugene,Information Not Available,datetime,"Eugene, OR, United States",...,10.0,2023-12-31T07:47:19,2023-12-31T16:43:40,0.66,Overcast,Cloudy skies throughout the day.,fog,"KCVO,WWCO3,72693024221,72694524202,KEUG",28.0,12.0
67519,2023-12-31,22:15:00,17,Chose not to disclose,White,English,Eugene,Information Not Available,datetime,"Eugene, OR, United States",...,10.0,2023-12-31T07:47:19,2023-12-31T16:43:40,0.66,Overcast,Cloudy skies throughout the day.,fog,"KCVO,WWCO3,72693024221,72694524202,KEUG",28.0,12.0


#### Turning 'Sunrise' and 'Sunset' into Integers

In [9]:
def replace_sunrise_sunset_with_hours(df):

    def extract_hour(datetime_str):
        if isinstance(datetime_str, str):
            return int(datetime_str[11:13])
        else:
            return None  # Handle missing or invalid values

    df['sunrise'] = df['sunrise'].apply(extract_hour)
    df['sunset'] = df['sunset'].apply(extract_hour)
    
    df.rename(columns={'sunrise': 'sunrise_hour', 'sunset': 'sunset_hour'}, inplace=True)
    
    return df

replace_sunrise_sunset_with_hours(raw_all_col_data)

,Date,TimeOfCall,Age,Gender,Race,Language,City,Reason for Dispatch,TimeOfCall_Type,name,...,severerisk,sunrise_hour,sunset_hour,moonphase,conditions,description,icon,stations,pm25,month
0,2021-01-04,20:10:31,48,Female,White,English,Eugene,Public Assist,time,"Eugene, OR, United States",...,NaN,7.0,16.0,0.70,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"KCVO,72693024221,99999904236,72694524202,F6274...",13.0,1.0
1,2021-01-14,14:28:00,34,Male,White,English,Springfield,Public Assist,time,"Eugene, OR, United States",...,NaN,7.0,16.0,0.05,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KCVO,WWCO3,72693024221,99999904236,72694524202...",24.0,1.0
2,2021-01-14,14:55:00,Unavailable,Chose not to Disclose,Chose not to Disclose,English,Springfield,Check Welfare,time,"Eugene, OR, United States",...,NaN,7.0,16.0,0.05,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KCVO,WWCO3,72693024221,99999904236,72694524202...",24.0,1.0
3,2021-01-14,14:53:00,Unavailable,Chose not to Disclose,Chose not to Disclose,English,Springfield,Check Welfare,time,"Eugene, OR, United States",...,NaN,7.0,16.0,0.05,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KCVO,WWCO3,72693024221,99999904236,72694524202...",24.0,1.0
4,2021-01-14,16:11:00,35,Male,White,English,Springfield,Check Welfare,time,"Eugene, OR, United States",...,NaN,7.0,16.0,0.05,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KCVO,WWCO3,72693024221,99999904236,72694524202...",24.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67516,2023-12-31,21:33:35,Not Given,Chose not to disclose,White,English,Springfield,Information Not Available,datetime,"Eugene, OR, United States",...,10.0,7.0,16.0,0.66,Overcast,Cloudy skies throughout the day.,fog,"KCVO,WWCO3,72693024221,72694524202,KEUG",28.0,12.0
67517,2023-12-31,21:51:35,Not Given,Chose not to disclose,NaN,English,Informatio not recorded,Information Not Available,datetime,"Eugene, OR, United States",...,10.0,7.0,16.0,0.66,Overcast,Cloudy skies throughout the day.,fog,"KCVO,WWCO3,72693024221,72694524202,KEUG",28.0,12.0
67518,2023-12-31,21:58:10,17,Chose not to disclose,White,English,Eugene,Information Not Available,datetime,"Eugene, OR, United States",...,10.0,7.0,16.0,0.66,Overcast,Cloudy skies throughout the day.,fog,"KCVO,WWCO3,72693024221,72694524202,KEUG",28.0,12.0
67519,2023-12-31,22:15:00,17,Chose not to disclose,White,English,Eugene,Information Not Available,datetime,"Eugene, OR, United States",...,10.0,7.0,16.0,0.66,Overcast,Cloudy skies throughout the day.,fog,"KCVO,WWCO3,72693024221,72694524202,KEUG",28.0,12.0


In [10]:
# Drop rows with missing values in 'Date' or 'TimeOfCall'
raw_all_col_data = raw_all_col_data.dropna(subset=['Date', 'TimeOfCall'])

# Drop rows where 'TimeOfCall' is of string type
raw_all_col_data = raw_all_col_data[raw_all_col_data['TimeOfCall'].apply(type) != str]

# Filter out rows with datetime values
time_values = raw_all_col_data[raw_all_col_data['TimeOfCall_Type'] == 'time']['TimeOfCall']

# Drop the column 'TimeOfCall_Type' in place
raw_all_col_data.drop(columns='TimeOfCall_Type', inplace=True)

# Convert time values to datetime format
raw_all_col_data.loc[time_values.index, 'TimeOfCall'] = pd.to_datetime(time_values, format='%H:%M:%S').dt.time

# Extract hour from 'TimeOfCall' and assign it to the 'Hour' column
raw_all_col_data['Hour'] = raw_all_col_data['TimeOfCall'].apply(lambda x: x.hour)

def extract_date_features(df, date_column_name):
    df[date_column_name] = pd.to_datetime(df[date_column_name])
    df['DayOfWeek'] = df[date_column_name].dt.dayofweek
    df['DayOfMonth'] = df[date_column_name].dt.day
    df['DayOfYear'] = df[date_column_name].dt.dayofyear
    df['Year'] = df[date_column_name].dt.year
    return df

# Creating dummy columns for date features
raw_all_col_data = extract_date_features(raw_all_col_data, 'Date')

# Define a function to map dates to seasons
def get_season(date):
    month = date.month
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

# Add a 'Season' column based on the 'Date' column
raw_all_col_data['Season'] = raw_all_col_data['Date'].apply(get_season)

# Add a 'Month' column that assigns the month as 1-12
raw_all_col_data['Month'] = pd.to_datetime(raw_all_col_data['Date']).dt.month

# Drop the 'TimeOfCall' column
raw_all_col_data.drop(columns=['Date', 'TimeOfCall'], inplace=True)

# Display the DataFrame
print(raw_all_col_data.columns)
raw_all_col_data

Index(['Age', 'Gender', 'Race', 'Language', 'City', 'Reason for Dispatch',
       'name', 'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'preciptype', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
       'solarenergy', 'uvindex', 'severerisk', 'sunrise_hour', 'sunset_hour',
       'moonphase', 'conditions', 'description', 'icon', 'stations', 'pm25',
       'month', 'Hour', 'DayOfWeek', 'DayOfMonth', 'DayOfYear', 'Year',
       'Season', 'Month'],
      dtype='object')


,Age,Gender,Race,Language,City,Reason for Dispatch,name,tempmax,tempmin,temp,...,stations,pm25,month,Hour,DayOfWeek,DayOfMonth,DayOfYear,Year,Season,Month
0,48,Female,White,English,Eugene,Public Assist,"Eugene, OR, United States",52.0,43.0,48.6,...,"KCVO,72693024221,99999904236,72694524202,F6274...",13.0,1.0,20,0,4,4,2021,Winter,1
1,34,Male,White,English,Springfield,Public Assist,"Eugene, OR, United States",53.8,38.0,44.5,...,"KCVO,WWCO3,72693024221,99999904236,72694524202...",24.0,1.0,14,3,14,14,2021,Winter,1
2,Unavailable,Chose not to Disclose,Chose not to Disclose,English,Springfield,Check Welfare,"Eugene, OR, United States",53.8,38.0,44.5,...,"KCVO,WWCO3,72693024221,99999904236,72694524202...",24.0,1.0,14,3,14,14,2021,Winter,1
3,Unavailable,Chose not to Disclose,Chose not to Disclose,English,Springfield,Check Welfare,"Eugene, OR, United States",53.8,38.0,44.5,...,"KCVO,WWCO3,72693024221,99999904236,72694524202...",24.0,1.0,14,3,14,14,2021,Winter,1
4,35,Male,White,English,Springfield,Check Welfare,"Eugene, OR, United States",53.8,38.0,44.5,...,"KCVO,WWCO3,72693024221,99999904236,72694524202...",24.0,1.0,16,3,14,14,2021,Winter,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67516,Not Given,Chose not to disclose,White,English,Springfield,Information Not Available,"Eugene, OR, United States",42.9,39.9,41.1,...,"KCVO,WWCO3,72693024221,72694524202,KEUG",28.0,12.0,21,6,31,365,2023,Winter,12
67517,Not Given,Chose not to disclose,NaN,English,Informatio not recorded,Information Not Available,"Eugene, OR, United States",42.9,39.9,41.1,...,"KCVO,WWCO3,72693024221,72694524202,KEUG",28.0,12.0,21,6,31,365,2023,Winter,12
67518,17,Chose not to disclose,White,English,Eugene,Information Not Available,"Eugene, OR, United States",42.9,39.9,41.1,...,"KCVO,WWCO3,72693024221,72694524202,KEUG",28.0,12.0,21,6,31,365,2023,Winter,12
67519,17,Chose not to disclose,White,English,Eugene,Information Not Available,"Eugene, OR, United States",42.9,39.9,41.1,...,"KCVO,WWCO3,72693024221,72694524202,KEUG",28.0,12.0,22,6,31,365,2023,Winter,12


In [11]:
grouped_data = raw_all_col_data['Year'].value_counts()
print(grouped_data)

Year
2022    23638
2021    23258
2023    19565
Name: count, dtype: int64


#### Data Cleaning (ignoring NaNs)

In [12]:
columns_to_drop = ['Language', 'stations', 'name', 'icon', 'moonphase', 'severerisk', 'winddir', 'description', 'preciptype', 'month']

raw_all_col_data = raw_all_col_data.drop(columns_to_drop, axis=1)

#### Cleaning 'Age'

In [13]:
# Replace 'Not Given' values with NA in the 'Age' column
raw_all_col_data['Age'] = raw_all_col_data['Age'].replace('Not Given', pd.NA)

# Replace 'Unavailable' values with NA in the 'Age' column
raw_all_col_data['Age'] = raw_all_col_data['Age'].replace('Unavailable', pd.NA)

#### Cleaning 'Gender'

In [14]:
# Convert all values in 'Gender' column to lowercase
raw_all_col_data['Gender'] = raw_all_col_data['Gender'].str.lower()

# Map similar values to a single representation
gender_mapping = {
    'female': 'female',
    'male': 'male',
    'chose not to disclose': 'undisclosed',
    'female (assigned male at birth)': 'trans_female',
    'male (assigned female at birth)': 'trans_male',
    'non-binary/ gender queer': 'non_binary'
}

raw_all_col_data['Gender'] = raw_all_col_data['Gender'].map(gender_mapping)

# Replace 'undisclosed' values with NA
raw_all_col_data['Gender'] = raw_all_col_data['Gender'].replace('undisclosed', pd.NA)

#### Cleaning 'Race'

In [15]:
# Convert all values in 'Race' column to lowercase
raw_all_col_data['Race'] = raw_all_col_data['Race'].str.lower()

# Map similar values to a single representation
race_mapping = {
    'alaska native': 'alaska native',
    'american indian': 'american indian',
    'american indian or alaska native': 'american indian/alaska native',
    'asian': 'asian',
    'black or african american': 'black/african american',
    'chose not to disclose': 'undisclosed',
    'hispanic or latino': 'hispanic/latino',
    'native hawaiian or other pac island': 'native hawaiian/other pacific islander',
    'other race': 'other',
    'other single race': 'other',
    'two or more unspecified race': 'two or more races',
    'white': 'white'
}

raw_all_col_data['Race'] = raw_all_col_data['Race'].map(race_mapping)

# Replace 'undisclosed' values with NA
raw_all_col_data['Race'] = raw_all_col_data['Race'].replace('undisclosed', pd.NA)

#### Cleaning 'City'

In [16]:
# Replace 'Informatio not recorded' values with NA in the 'City' column
raw_all_col_data['City'] = raw_all_col_data['City'].replace('Informatio not recorded', pd.NA)

#### Cleaning 'Reason for Dispatch'

In [17]:
# Replace 'Information Not Available' values with NA in the 'Reason for Dispatch' column
raw_all_col_data['Reason for Dispatch'] = raw_all_col_data['Reason for Dispatch'].replace('Information Not Available', pd.NA)

#### Checking for Errors

In [18]:
# Print unique values in the 'Age' column
print("Unique values in 'Age' column:")
print(raw_all_col_data['Age'].unique())

# Print unique values in the 'Gender' column
print("\nUnique values in 'Gender' column:")
print(raw_all_col_data['Gender'].unique())

# Print unique values in the 'Race' column
print("\nUnique values in 'Race' column:")
print(raw_all_col_data['Race'].unique())

# Print unique values in the 'City' column
print("\nUnique values in 'City' column:")
print(raw_all_col_data['City'].unique())

# Print unique values in the 'Reason for Dispatch' column
print("\nUnique values in 'Reason for Dispatch' column:")
print(raw_all_col_data['Reason for Dispatch'].unique())

Unique values in 'Age' column:
[48 34 <NA> 35 68 72 21 31 22 76 58 56 20 32 45 29 36 28 55 18 49 73 94 12
 60 17 46 59 44 69 40 24 50 30 63 19 7 25 64 26 16 54 39 66 57 62 84 85 70
 82 15 43 67 33 65 75 11 13 81 23 38 14 53 42 41 79 52 51 27 61 37 78 47
 87 77 80 83 92 6 9 71 90 5 10 89 74 91 96 8 95 93 86 88 97 4 98]

Unique values in 'Gender' column:
['female' 'male' <NA> nan 'trans_female' 'trans_male' 'non_binary']

Unique values in 'Race' column:
['white' <NA> 'black/african american' 'two or more races'
 'american indian' 'asian' 'other'
 'native hawaiian/other pacific islander' 'alaska native' nan
 'american indian/alaska native' 'hispanic/latino']

Unique values in 'City' column:
['Eugene' 'Springfield' nan <NA>]

Unique values in 'Reason for Dispatch' column:
['Public Assist' 'Check Welfare' 'EMS Assist' 'Police Assist' 'Transport'
 'Counseling' 'Suicidal Subject' 'Fire Assist' <NA>]


In [19]:
# Create a copy of raw_all_col_data and name it clean_data
clean_data = raw_all_col_data.copy()
clean_data

,Age,Gender,Race,City,Reason for Dispatch,tempmax,tempmin,temp,feelslikemax,feelslikemin,...,sunset_hour,conditions,pm25,Hour,DayOfWeek,DayOfMonth,DayOfYear,Year,Season,Month
0,48,female,white,Eugene,Public Assist,52.0,43.0,48.6,52.0,37.8,...,16.0,"Rain, Partially cloudy",13.0,20,0,4,4,2021,Winter,1
1,34,male,white,Springfield,Public Assist,53.8,38.0,44.5,53.8,37.7,...,16.0,Partially cloudy,24.0,14,3,14,14,2021,Winter,1
2,<NA>,<NA>,<NA>,Springfield,Check Welfare,53.8,38.0,44.5,53.8,37.7,...,16.0,Partially cloudy,24.0,14,3,14,14,2021,Winter,1
3,<NA>,<NA>,<NA>,Springfield,Check Welfare,53.8,38.0,44.5,53.8,37.7,...,16.0,Partially cloudy,24.0,14,3,14,14,2021,Winter,1
4,35,male,white,Springfield,Check Welfare,53.8,38.0,44.5,53.8,37.7,...,16.0,Partially cloudy,24.0,16,3,14,14,2021,Winter,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67516,<NA>,<NA>,white,Springfield,<NA>,42.9,39.9,41.1,42.9,37.2,...,16.0,Overcast,28.0,21,6,31,365,2023,Winter,12
67517,<NA>,<NA>,NaN,<NA>,<NA>,42.9,39.9,41.1,42.9,37.2,...,16.0,Overcast,28.0,21,6,31,365,2023,Winter,12
67518,17,<NA>,white,Eugene,<NA>,42.9,39.9,41.1,42.9,37.2,...,16.0,Overcast,28.0,21,6,31,365,2023,Winter,12
67519,17,<NA>,white,Eugene,<NA>,42.9,39.9,41.1,42.9,37.2,...,16.0,Overcast,28.0,22,6,31,365,2023,Winter,12


In [20]:
def create_dummies(data):
    # Get list of qualitative (categorical) columns excluding 'Age' if it exists
    if 'Age' in data.columns:
        qualitative_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()
        qualitative_cols.remove('Age')
    else:
        qualitative_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()

    # Replace missing values in qualitative columns with 'missing'
    data[qualitative_cols] = data[qualitative_cols].fillna('missing')

    # Create dummy variables for every unique value in every qualitative column
    dummy_columns = []
    for col in qualitative_cols:
        dummies = pd.get_dummies(data[col], prefix=col)
        dummy_columns.extend(dummies.columns.tolist())
        data = pd.concat([data, dummies], axis=1)

        # Set dummy variables to NA if the _missing column is True
        if f"{col}_missing" in data.columns:
            missing_col = f"{col}_missing"
            data.loc[data[missing_col] == 1, dummies.columns] = pd.NA

        # Drop _missing column
        if f"{col}_missing" in data.columns:
            data.drop(columns=[f"{col}_missing"], inplace=True)

    # Drop original qualitative columns
    data.drop(columns=qualitative_cols, inplace=True)

    return data


# Assuming clean_data is your DataFrame
clean_data_with_dummies = create_dummies(clean_data)
print(clean_data_with_dummies.columns)
clean_data_with_dummies

/var/folders/bl/bhy6kgn16kx9r1j535zcctcw0000gn/T/ipykernel_19120/3263728760.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  data.loc[data[missing_col] == 1, dummies.columns] = pd.NA
/var/folders/bl/bhy6kgn16kx9r1j535zcctcw0000gn/T/ipykernel_19120/3263728760.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  data.loc[data[missing_col] == 1, dummies.columns] = pd.NA
/var/folders/bl/bhy6kgn16kx9r1j535zcctcw0000gn/T/ipykernel_19120/3263728760.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  da

Index(['Age', 'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'snow', 'snowdepth', 'windgust', 'windspeed', 'sealevelpressure',
       'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex',
       'sunrise_hour', 'sunset_hour', 'pm25', 'Hour', 'DayOfWeek',
       'DayOfMonth', 'DayOfYear', 'Year', 'Month', 'Gender_female',
       'Gender_male', 'Gender_non_binary', 'Gender_trans_female',
       'Gender_trans_male', 'Race_alaska native', 'Race_american indian',
       'Race_american indian/alaska native', 'Race_asian',
       'Race_black/african american', 'Race_hispanic/latino',
       'Race_native hawaiian/other pacific islander', 'Race_other',
       'Race_two or more races', 'Race_white', 'City_Eugene',
       'City_Springfield', 'Reason for Dispatch_Check Welfare',
       'Reason for Dispatch_Counseling', 'Reason for Dispatch_EMS Assist',
       'Reason for Dispatch_F

/var/folders/bl/bhy6kgn16kx9r1j535zcctcw0000gn/T/ipykernel_19120/3263728760.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  data.loc[data[missing_col] == 1, dummies.columns] = pd.NA


,Age,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,"conditions_Rain, Freezing Drizzle/Freezing Rain, Partially cloudy","conditions_Rain, Overcast","conditions_Rain, Partially cloudy","conditions_Snow, Rain","conditions_Snow, Rain, Overcast","conditions_Snow, Rain, Partially cloudy",Season_Autumn,Season_Spring,Season_Summer,Season_Winter
0,48,52.0,43.0,48.6,52.0,37.8,46.3,43.8,83.6,0.823,...,False,False,True,False,False,False,False,False,False,True
1,34,53.8,38.0,44.5,53.8,37.7,43.5,40.1,85.0,0.000,...,False,False,False,False,False,False,False,False,False,True
2,<NA>,53.8,38.0,44.5,53.8,37.7,43.5,40.1,85.0,0.000,...,False,False,False,False,False,False,False,False,False,True
3,<NA>,53.8,38.0,44.5,53.8,37.7,43.5,40.1,85.0,0.000,...,False,False,False,False,False,False,False,False,False,True
4,35,53.8,38.0,44.5,53.8,37.7,43.5,40.1,85.0,0.000,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67516,<NA>,42.9,39.9,41.1,42.9,37.2,39.8,39.8,95.1,0.000,...,False,False,False,False,False,False,False,False,False,True
67517,<NA>,42.9,39.9,41.1,42.9,37.2,39.8,39.8,95.1,0.000,...,False,False,False,False,False,False,False,False,False,True
67518,17,42.9,39.9,41.1,42.9,37.2,39.8,39.8,95.1,0.000,...,False,False,False,False,False,False,False,False,False,True
67519,17,42.9,39.9,41.1,42.9,37.2,39.8,39.8,95.1,0.000,...,False,False,False,False,False,False,False,False,False,True


#### Cleaning the Upcoming Data

In [21]:
columns_to_drop = ['stations', 'name', 'icon', 'moonphase', 'severerisk', 'winddir', 'description', 'preciptype']

upcoming_data = upcoming_data.drop(columns_to_drop, axis=1)
upcoming_data

,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,sunrise,sunset,conditions,pm25
0,2024-06-04,69.0,53.0,60.2,69.0,53.0,60.2,52.8,77.0,0.000,...,1016.5,79.1,10.1,91.9,8.0,4,2024-06-04T05:30:47,2024-06-04T20:51:22,Partially cloudy,16.47037
1,2024-06-05,76.0,53.0,62.2,76.0,53.0,62.2,51.4,71.0,0.000,...,1020.5,34.7,10.1,329.5,28.5,9,2024-06-05T05:30:25,2024-06-05T20:52:05,Partially cloudy,16.47037
2,2024-06-06,80.0,49.0,64.6,80.0,49.0,64.6,48.5,58.9,0.000,...,1015.8,4.0,10.1,357.1,31.0,10,2024-06-06T05:30:06,2024-06-06T20:52:45,Clear,16.47037
3,2024-06-07,84.3,53.7,68.8,83.4,53.7,68.8,54.2,61.9,0.000,...,1011.7,9.5,15.0,350.0,30.1,9,2024-06-07T05:29:48,2024-06-07T20:53:24,Clear,16.47037
4,2024-06-08,78.9,54.9,67.1,78.9,54.9,67.1,52.9,62.3,0.067,...,1016.3,40.8,15.0,323.9,28.0,9,2024-06-08T05:29:33,2024-06-08T20:54:01,Partially cloudy,16.47037
5,2024-06-09,75.5,54.2,64.5,75.5,54.2,64.5,51.6,64.6,0.020,...,1017.4,29.3,15.0,352.5,30.5,9,2024-06-09T05:29:20,2024-06-09T20:54:37,Partially cloudy,16.47037
6,2024-06-10,76.7,51.5,64.2,76.7,51.5,64.2,49.4,61.2,0.004,...,1016.0,7.2,15.0,365.1,31.7,10,2024-06-10T05:29:10,2024-06-10T20:55:10,Clear,16.47037
7,2024-06-11,77.3,50.1,61.8,77.3,50.1,61.8,50.3,67.0,0.000,...,1017.3,9.7,15.0,337.0,29.1,9,2024-06-11T05:29:01,2024-06-11T20:55:42,Clear,16.47037


In [22]:
# Creating dummy columns for date features
upcoming_data = extract_date_features(upcoming_data, 'date')

# Add a 'Season' column based on the 'Date' column
upcoming_data['Season'] = upcoming_data['date'].apply(get_season)

# Add a 'Month' column that assigns the month as 1-12
upcoming_data['Month'] = pd.to_datetime(upcoming_data['date']).dt.month

# Replacing sunrise/sunset times with integers
replace_sunrise_sunset_with_hours(upcoming_data)

# Creating dummy variables
upcoming_data_with_dummies = create_dummies(upcoming_data)

def align_dummies_columns(df1, df2):
    # Get the set of dummy variable columns from both DataFrames
    df1_dummy_cols = set(df1.columns)
    df2_dummy_cols = set(df2.columns)

    # Find missing columns in each DataFrame
    missing_cols_df1 = df2_dummy_cols - df1_dummy_cols
    missing_cols_df2 = df1_dummy_cols - df2_dummy_cols

    # Add missing columns to each DataFrame and fill them with False
    for col in missing_cols_df1:
        df1[col] = False
    for col in missing_cols_df2:
        df2[col] = False

    # Reorder columns to match between DataFrames
    df1 = df1.reindex(sorted(df2.columns), axis=1)
    df2 = df2.reindex(sorted(df1.columns), axis=1)

    return df1, df2

# Creating dummies 
upcoming_data_with_dummies = create_dummies(upcoming_data)

# Adding an missing dummy columns
clean_data_with_dummies, upcoming_data_with_dummies = align_dummies_columns(clean_data_with_dummies, upcoming_data_with_dummies)
upcoming_data_with_dummies.columns

Index(['Age', 'City_Eugene', 'City_Springfield', 'DayOfMonth', 'DayOfWeek',
       'DayOfYear', 'Gender_female', 'Gender_male', 'Gender_non_binary',
       'Gender_trans_female', 'Gender_trans_male', 'Hour', 'Month',
       'Race_alaska native', 'Race_american indian',
       'Race_american indian/alaska native', 'Race_asian',
       'Race_black/african american', 'Race_hispanic/latino',
       'Race_native hawaiian/other pacific islander', 'Race_other',
       'Race_two or more races', 'Race_white',
       'Reason for Dispatch_Check Welfare', 'Reason for Dispatch_Counseling',
       'Reason for Dispatch_EMS Assist', 'Reason for Dispatch_Fire Assist',
       'Reason for Dispatch_Police Assist',
       'Reason for Dispatch_Public Assist',
       'Reason for Dispatch_Suicidal Subject', 'Reason for Dispatch_Transport',
       'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
       'Year', 'cloudcover', 'conditions_Clear', 'conditions_Overcast',
       'conditions_Partia

In [23]:
columns_to_drop = ['Age', 'City_Eugene', 'City_Springfield', 'Gender_female', 'Gender_male', 'Gender_non_binary',
       'Gender_trans_female', 'Gender_trans_male', 'Hour', 'Race_alaska native', 'Race_american indian',
       'Race_american indian/alaska native', 'Race_asian',
       'Race_black/african american', 'Race_hispanic/latino',
       'Race_native hawaiian/other pacific islander', 'Race_other',
       'Race_two or more races', 'Race_white',
       'Reason for Dispatch_Check Welfare', 'Reason for Dispatch_Counseling',
       'Reason for Dispatch_EMS Assist', 'Reason for Dispatch_Fire Assist',
       'Reason for Dispatch_Police Assist',
       'Reason for Dispatch_Public Assist',
       'Reason for Dispatch_Suicidal Subject', 'Reason for Dispatch_Transport', 'date']

upcoming_data_with_dummies = upcoming_data_with_dummies.drop(columns_to_drop, axis=1)
upcoming_data_with_dummies

,DayOfMonth,DayOfWeek,DayOfYear,Month,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,Year,cloudcover,...,solarradiation,sunrise_hour,sunset_hour,temp,tempmax,tempmin,uvindex,visibility,windgust,windspeed
0,4,1,156,6,False,False,True,False,2024,79.1,...,91.9,5,20,60.2,69.0,53.0,4,10.1,16.1,6.9
1,5,2,157,6,False,False,True,False,2024,34.7,...,329.5,5,20,62.2,76.0,53.0,9,10.1,15.0,8.1
2,6,3,158,6,False,False,True,False,2024,4.0,...,357.1,5,20,64.6,80.0,49.0,10,10.1,16.1,8.1
3,7,4,159,6,False,False,True,False,2024,9.5,...,350.0,5,20,68.8,84.3,53.7,9,15.0,11.6,6.3
4,8,5,160,6,False,False,True,False,2024,40.8,...,323.9,5,20,67.1,78.9,54.9,9,15.0,13.2,7.2
5,9,6,161,6,False,False,True,False,2024,29.3,...,352.5,5,20,64.5,75.5,54.2,9,15.0,15.2,7.4
6,10,0,162,6,False,False,True,False,2024,7.2,...,365.1,5,20,64.2,76.7,51.5,10,15.0,15.0,7.4
7,11,1,163,6,False,False,True,False,2024,9.7,...,337.0,5,20,61.8,77.3,50.1,9,15.0,14.3,8.9


In [24]:
# Define a function to check if a value is either NaN, <NA>, TRUE, FALSE, or a number
def check_value(x):
    return pd.isna(x) or pd.isna(pd.NA) or isinstance(x, bool) or pd.api.types.is_numeric_dtype(x)

# Apply the function to every value in the dataframe
def check_dataframe(df):
    result = df.applymap(check_value)

    # Check if all values are True (indicating they are one of the specified types)
    all_values_valid = result.all().all()

    return all_values_valid

# Check 'clean_data_with_dummies'
all_values_valid_clean = check_dataframe(clean_data_with_dummies)
print("All values in 'clean_data_with_dummies' are either NaN, <NA>, TRUE, FALSE, or a number:", all_values_valid_clean)

# Check 'upcoming_data_with_dummies'
all_values_valid_upcoming = check_dataframe(upcoming_data_with_dummies)
print("All values in 'upcoming_data_with_dummies' are either NaN, <NA>, TRUE, FALSE, or a number:", all_values_valid_upcoming)

/var/folders/bl/bhy6kgn16kx9r1j535zcctcw0000gn/T/ipykernel_19120/293450811.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result = df.applymap(check_value)


All values in 'clean_data_with_dummies' are either NaN, <NA>, TRUE, FALSE, or a number: True
All values in 'upcoming_data_with_dummies' are either NaN, <NA>, TRUE, FALSE, or a number: True


/var/folders/bl/bhy6kgn16kx9r1j535zcctcw0000gn/T/ipykernel_19120/293450811.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result = df.applymap(check_value)


In [25]:
# Save the cleaned 3-year data
clean_data_with_dummies.to_csv('data/clean_data_with_dummies.csv', index=False)

In [26]:
# Save the cleaned upcoming data
upcoming_data_with_dummies.to_csv('data/upcoming_data_with_dummies.csv', index=False)